In [1]:
# Initial imports
import os
import pandas as pd
import requests
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Reading the News API key enviroment variable
api_key = os.getenv("crypto_news_key")

In [3]:
print(f"API key:{type(api_key)}")

API key:<class 'str'>


In [4]:
#Retrieve the latest news events and eventIDs. Events are important headlines that are receiving a significant amount of news coverage.
crypto_events = 'https://cryptonews-api.com/api/v1/events?&token='+ api_key

In [5]:
response_crypto_events = requests.get(crypto_events)

In [6]:
response_crypto_events

<Response [200]>

In [7]:
raw_crypto_events = response_crypto_events.json()


In [8]:
raw_crypto_events

{'data': [{'event_name': 'Ripple Asks Court to Force SEC to Produce Documents Regarding Bitcoin and Ethereum',
   'event_id': 'B100',
   'news_items': 4,
   'date': 'Tue, 16 Mar 2021 12:00:00 -0400',
   'tickers': ['XRP', 'BTC', 'ETH']},
  {'event_name': 'Bitcoin falls after weekend rally tops $61K',
   'event_id': 'B99',
   'news_items': 10,
   'date': 'Mon, 15 Mar 2021 20:26:00 -0400',
   'tickers': ['BTC']},
  {'event_name': 'Americans could spend $40 Billion in stimulus on bitcoin',
   'event_id': 'B98',
   'news_items': 7,
   'date': 'Mon, 15 Mar 2021 18:00:00 -0400',
   'tickers': ['BTC']},
  {'event_name': 'Another DeFi Hack: PancakeSwap, Cream Finance Websites Compromised',
   'event_id': 'B97',
   'news_items': 11,
   'date': 'Mon, 15 Mar 2021 12:59:00 -0400',
   'tickers': ['CAKE', 'CREAM']},
  {'event_name': 'India To Introduce All Out Cryptocurrency Ban Targeting Miners and Traders: Report',
   'event_id': 'B96',
   'news_items': 13,
   'date': 'Mon, 15 Mar 2021 07:00:00 -0

In [18]:
events_df = pd.DataFrame(data = raw_crypto_events['data'])

In [19]:
events_df.head()

,event_name,event_id,news_items,date,tickers
0,Ripple Asks Court to Force SEC to Produce Docu...,B100,4,"Tue, 16 Mar 2021 12:00:00 -0400","[XRP, BTC, ETH]"
1,Bitcoin falls after weekend rally tops $61K,B99,10,"Mon, 15 Mar 2021 20:26:00 -0400",[BTC]
2,Americans could spend $40 Billion in stimulus ...,B98,7,"Mon, 15 Mar 2021 18:00:00 -0400",[BTC]
3,"Another DeFi Hack: PancakeSwap, Cream Finance ...",B97,11,"Mon, 15 Mar 2021 12:59:00 -0400","[CAKE, CREAM]"
4,India To Introduce All Out Cryptocurrency Ban ...,B96,13,"Mon, 15 Mar 2021 07:00:00 -0400",[]


In [17]:
# Create the ETH sentiment scores DataFrame
sentiments = []

for data in raw_crypto_events['data']:
    try:
        date = data['date']
        text = data["event_name"]
        results = analyzer.polarity_scores(text)
        compound = results["compound"]
        tickers = data['tickers']
        pos = results["pos"]
        neu = results["neu"]
        neg = results["neg"]

        sentiments.append({
            "date": date,
            "text": text,
            "tickers":tickers,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
        })
    except AttributeError:
        pass
    
eth  = pd.DataFrame(sentiments)


In [18]:
eth.head()

,date,text,tickers,Compound,Positive,Negative,Neutral
0,"Tue, 16 Mar 2021 12:00:00 -0400",Ripple Asks Court to Force SEC to Produce Docu...,"[XRP, BTC, ETH]",0.0000,0.000,0.000,1.000
1,"Mon, 15 Mar 2021 20:26:00 -0400",Bitcoin falls after weekend rally tops $61K,[BTC],0.5106,0.355,0.000,0.645
2,"Mon, 15 Mar 2021 18:00:00 -0400",Americans could spend $40 Billion in stimulus ...,[BTC],0.0000,0.000,0.000,1.000
3,"Mon, 15 Mar 2021 12:59:00 -0400","Another DeFi Hack: PancakeSwap, Cream Finance ...","[CAKE, CREAM]",0.0000,0.000,0.000,1.000
4,"Mon, 15 Mar 2021 07:00:00 -0400",India To Introduce All Out Cryptocurrency Ban ...,[],-0.5574,0.000,0.247,0.753


In [19]:
eth.describe()

,Compound,Positive,Negative,Neutral
count,50.000000,50.000000,50.00000,50.000000
mean,0.010926,0.061220,0.03914,0.899640
std,0.236178,0.123341,0.09423,0.138572
min,-0.599400,0.000000,0.00000,0.417000
25%,0.000000,0.000000,0.00000,0.789000
50%,0.000000,0.000000,0.00000,1.000000
75%,0.000000,0.000000,0.00000,1.000000
max,0.718400,0.583000,0.35800,1.000000
